In [2]:
import numpy as np
import pandas as pd
from surprise import Reader, Dataset, accuracy, SVD
from surprise.model_selection import cross_validate, train_test_split
from sklearn.metrics import ndcg_score
from recommenders.evaluation.python_evaluation import ndcg_at_k
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import ndcg_score

In [3]:
dataframe = pd.read_csv('training_set_VU_DM.csv')
# create score variable
conditions = [
    (dataframe['booking_bool'] == 1),
    (dataframe['booking_bool'] == 0) & (dataframe['click_bool'] == 1),
    (dataframe['booking_bool'] == 0) & (dataframe['click_bool'] == 0)
]

choices = [5, 1, 0]

dataframe['score'] = np.select(conditions, choices, default=np.nan)

In [4]:
dataframe = dataframe[['srch_id','prop_id','score']].copy()

In [5]:
dataframe[dataframe['srch_id']==11]

,srch_id,prop_id,score
86,11,6492,0.0
87,11,8946,0.0
88,11,9952,0.0
89,11,19470,0.0
90,11,20499,1.0
91,11,24526,0.0
92,11,26421,0.0
93,11,27675,0.0
94,11,28364,0.0
95,11,33980,0.0


In [6]:
# Extract the 'srch_id' column as the groups
groups = dataframe['srch_id']

# Create a GroupShuffleSplit object
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# Perform the group shuffle split
train_indices, test_indices = next(gss.split(dataframe, groups=groups))

# Split the data into train and test sets
dataframe_trainset = dataframe.iloc[train_indices]
dataframe_testset = dataframe.iloc[test_indices]

In [7]:
train = Dataset.load_from_df(dataframe_trainset, reader=Reader(rating_scale=(0,1))) # turn into Surprise dataset
test = Dataset.load_from_df(dataframe_testset, reader=Reader(rating_scale=(0,1))) 

train = train.build_full_trainset()
test = test.build_full_trainset().build_testset()

# trainset, testset = train_test_split(df, test_size=0.2, random_state=22) # Split the data
algo = SVD() # Define svd algorithm
algo.fit(train) # Train the model
predictions = algo.test(test) # Make predictions

In [43]:
data = {
    'srch_id': [pred.uid for pred in predictions],
    'prop_id': [pred.iid for pred in predictions],
    'score': [pred.r_ui for pred in predictions],
    'prediction': [pred.est for pred in predictions]
}
df = pd.DataFrame(data)
df

,srch_id,prop_id,score,prediction
0,11,6492,0.0,0.197036
1,11,8946,0.0,0.042003
2,11,9952,0.0,0.076565
3,11,19470,0.0,0.098746
4,11,20499,1.0,0.102853
...,...,...,...,...
991660,332784,114177,0.0,0.217913
991661,332784,115562,0.0,0.163295
991662,332784,121275,0.0,0.179538
991663,332784,128677,0.0,0.258813


In [44]:
df_true = df[['srch_id', 'score']].copy().sort_values(by=['srch_id'], ascending=[True])
df_pred = df[['srch_id', 'prediction']].copy().sort_values(by=['srch_id'], ascending=[True])

In [45]:
range(len(df_true))

range(0, 991665)

In [52]:
# group_counts = df.groupby('srch_id').value_counts()
y_true = df.groupby('srch_id')['score'].apply(lambda x: x.values).values
y_pred = df.groupby('srch_id')['score'].apply(lambda x: x.values).values
print(y_true)


[array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
 array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.])
 ...
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 5., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
 array([0., 0., 0., 5., 0., 0., 0., 0.])
 array([0., 0., 0., 0., 0., 0., 0., 0., 5., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])                       ]


In [58]:
y_true.values

AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [56]:
ndcg_scores = []
for value in range(len(y_true)):
    ndcg_scores.append(ndcg_score(y_true[value], y_pred[value], k=5))

ValueError: Only ('multilabel-indicator', 'continuous-multioutput', 'multiclass-multioutput') formats are supported. Got binary instead

In [16]:
for value in df['srch_id'].unique():
    group = df.loc[df['srch_id'].isin(value)]

TypeError: only list-like objects are allowed to be passed to isin(), you passed a [int64]

In [17]:
group = df.loc[df['srch_id'].isin(value)]

TypeError: only list-like objects are allowed to be passed to isin(), you passed a [int64]

In [11]:
# Example y_true and y_score arrays
y_true = np.array([[1, 0, 0], [0, 5, 0], [0, 0, 1]])
y_score = np.array([[0.1, 0.2, 0.2], [0.1, 0.0, 0.1], [0.2, 0.2, 0.1]])

# Calculate nDCG@5 score
ndcg5 = ndcg_score(y_true, y_score, k=5)

print("nDCG@5 score:", ndcg5)

nDCG@5 score: 0.5


In [12]:
y_true

array([[1, 0, 0],
       [0, 5, 0],
       [0, 0, 1]])

In [13]:
eval_ndcg = ndcg_at_k(df_true, df_pred, col_user='srch_id', col_item='prop_id', col_rating='score', col_prediction='prediction', 
                      k=5)
eval_ndcg

Missing column: prop_id in DataFrame


ValueError: Missing columns in true rating DataFrame